# [ Karate Dataset Classification with MLP ]
- embedded with LINE (first-order proximity / negative sampling )

# 1. Importing libraries & dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
ev = pd.read_csv('[Karate]Embedded_with_FirstOrder.csv')

In [3]:
data = ev[['X','Y','Color']]
data.columns = ['x1','x2','class']
data = data.sample(frac=1) # to shuffle

data.head()

,x1,x2,class
2,-1.703764,-1.234125,1.0
6,-0.772868,-0.608659,1.0
13,-0.730921,-0.467946,1.0
21,0.297496,-0.370970,1.0
5,-0.598505,-0.614123,1.0


# 2. Define Functions

### Basic functions
- 1) train_test_split
- 2) standard scaler
- 3) transpose & matrix multiplication

In [4]:
def train_test_split(data,test_ratio):
    #data.iloc[:,[0,1]] = standard_scaler(data.iloc[:,[0,1]])
    test_index = np.random.choice(len(data),int(len(data)*test_ratio),replace=False)
    train = data[~data.index.isin(test_index)]
    test = data[data.index.isin(test_index)]
    
    train_X = np.array(train)[:,[0,1]]
    train_y = np.array(train)[:,[2]].flatten()
    train_y = np.column_stack((1-train_y,train_y))
    
    test_X = np.array(test)[:,[0,1]]
    test_y = np.array(test)[:,[2]].flatten()
    test_y = np.column_stack((1-test_y,test_y))
    return train_X,train_y, test_X,test_y

In [5]:
def standard_scaler(x):
    mean = np.mean(x)
    std = np.std(x)
    return (x-mean)/std

In [6]:
def _t(X):
    return np.transpose(X)

def _m(A,B):
    return np.matmul(A,B)

### Activation functions
- 1) Sigmoid
- 2) Softmax

In [7]:
class Sigmoid:
    def __init__(self):
        self.last_o = 1
    
    def __call__(self,X):
        self.last_o = 1/(1+np.exp(-X))
        return self.last_o
    
    def grad(self):
        return self.last_o*(1-self.last_o)

In [8]:
class Softmax:
    def __init__(self):
        self.last_o = 1
        
    def __call__(self,X):
        e_x = np.exp(X-np.max(X))
        self.last_o = e_x / e_x.sum()
        return self.last_o
    
    def grad(self):
        return self.last_o*(1-self.last_o)

### Loss Function

In [9]:
class LogLoss:
    def __init__(self):
        self.dh = 1
        self.last_diff = 1
    
    def __call__(self,y,yhat):
        self.last_diff = yhat-y
        total_loss = np.mean(y*np.log(yhat+(1e-5)) + (1-y)*np.log(1-yhat+(1e-5)))
        return -total_loss
    
    def grad(self):
        return self.last_diff

# 3. Network Architecture

### 1) Neuron

In [10]:
class Neuron :
    def __init__(self,W,b,activation):
        self.W = W
        self.b = b
        self.act= activation()
        
        self.dW = np.zeros_like(self.W)  
        self.db = np.zeros_like(self.b)
        self.dh = np.zeros_like(_t(self.W)) 
        
        self.last_x = np.zeros((self.W.shape[0])) 
        self.last_h = np.zeros((self.W.shape[1]))
        
    def __call__(self,x):
        self.last_x = x
        self.last_h = _m(_t(self.W),x) + self.b
        output = self.act(self.last_h)
        return output
    
    def grad(self): 
        grad = self.act.grad()*self.W
        return grad
    
    def grad_W(self,dh): 
        grad = np.ones_like(self.W) 
        grad_a = self.act.grad()   # dh/du     
        for j in range(grad.shape[1]):
            grad[:,j] = dh[j] * grad_a[j] * self.last_x     # previous gradient * dh/du * du/dW
        return grad
        
    def grad_b(self,dh) : # dh/db = dh/du * du/db
        grad = dh * self.act.grad() * 1  # previous gradient * dh/du * du/db
        return grad

### 2) Neural Network

In [11]:
class NN:
    def __init__(self,input_num,output_num,hidden_depth,num_neuron, 
                 activation=Sigmoid, activation2=Softmax): 
        def init_var(in_,out_):
            weight = np.random.uniform(-5,5,(in_,out_))
            bias = np.zeros((out_,))
            return weight,bias
           
    ## 1-1. Hidden Layer
        self.sequence = list() # lists to put neurons
        W,b = init_var(input_num,num_neuron)
        self.sequence.append(Neuron(W,b,activation))
    
        for _ in range(hidden_depth-1):
            W,b = init_var(num_neuron,num_neuron)
            self.sequence.append(Neuron(W,b,activation)) # default : Sigmoid
    
    ## 1-2. Output Layer
        W,b = init_var(num_neuron,output_num)
        self.sequence.append(Neuron(W,b,activation2)) # default : Softmax
    
    def __call__(self,x):
        for layer in self.sequence:
            x = layer(x)
        return x
    
    def calc_grad(self,loss_fun):
        loss_fun.dh = loss_fun.grad()
        self.sequence.append(loss_fun)
        
        for i in range(len(self.sequence)-1, 0, -1):
            L1 = self.sequence[i]
            L0 = self.sequence[i-1]
            
            L0.dh = _m(L0.grad(), L1.dh)
            L0.dW = L0.grad_W(L1.dh)
            L0.db = L0.grad_b(L1.dh)
            
        self.sequence.remove(loss_fun)   

### 3) Gradient Descent

In [12]:
def GD(nn,x,y,loss_fun,lr=0.01):
    loss = loss_fun(nn(x),y) # 1) FEED FORWARD
    nn.calc_grad(loss_fun) # 2) BACK PROPAGATION
    
    for layer in nn.sequence: # Update Equation
        layer.W += -lr*layer.dW
        layer.b += -lr*layer.db    
    return loss

# 4. Train Model

In [13]:
train_X_70, train_y_70, test_X_70, test_y_70 = train_test_split(data,0.3)

In [19]:
NeuralNet_70 = NN(2,2,3,3,activation=Sigmoid, activation2=Softmax) # input_num, output_num, hidden_depth, num_layers
loss_fun = LogLoss()
EPOCH = 1000

loss_per_epoch_70 = []

for epoch in range(EPOCH):
    for i in range(train_X_70.shape[0]):
        loss = GD(NeuralNet_70,train_X_70[i],train_y_70[i],loss_fun,0.0001)
    loss_per_epoch_70.append(loss)
    if epoch%50 ==0:
        print('Epoch {} : Loss {}'.format(epoch+1, loss))

Epoch 1 : Loss 5.63392552538397
Epoch 51 : Loss 5.590625718799993
Epoch 101 : Loss 5.54521797399155
Epoch 151 : Loss 5.497608803499927
Epoch 201 : Loss 5.447702894709621
Epoch 251 : Loss 5.395403601608994
Epoch 301 : Loss 5.340613560209958
Epoch 351 : Loss 5.283235446507889
Epoch 401 : Loss 5.223172897185175
Epoch 451 : Loss 5.160331614153359
Epoch 501 : Loss 5.094620674258324
Epoch 551 : Loss 5.025954064729632
Epoch 601 : Loss 4.954252462880785
Epoch 651 : Loss 4.8794452747085
Epoch 701 : Loss 4.801472940919028
Epoch 751 : Loss 4.720289509985445
Epoch 801 : Loss 4.635865465606903
Epoch 851 : Loss 4.5481907799584285
Epoch 901 : Loss 4.457278144141624
Epoch 951 : Loss 4.363166303323437


# 5. Prediction

In [20]:
def predict(model,test_X):
    preds = []
    for i in range(test_X.shape[0]):
        pred_result = np.argmax(model(test_X[i]))
        preds.append(pred_result)
    return np.array(preds)

### 1) prediction result

In [21]:
pred70 = predict(NeuralNet_70,test_X_70)

In [22]:
test_y_70

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [24]:
pred70

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 1], dtype=int64)

### 2) metrics

In [25]:
def Metrics(pred,actual):
    TP,TN,FP,FN = 0,0,0,0
    for i in range(len(pred)):
        if pred[i]*actual[i]==1:
            TP +=1
        elif pred[i]>actual[i]:
            FP +=1
        elif pred[i]<actual[i]:
            FN +=1
        else:
            TN +=1
    
    accuracy = (TP+TN) / (TP+TN+FP+FN)
    precision = TP / (TP+FP)
    recall = TP / (TP+FN)
    F1_score = 2*(precision*recall)/(precision+recall)
    return accuracy,precision,recall,F1_score

In [26]:
print('Training Dataset 70%')
actual_class_70 = (1-test_y_70)[:,0]
Metrics(pred70,actual_class_70)

Training Dataset 70%


(0.8, 0.75, 1.0, 0.8571428571428571)